In [38]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import numpy as np

# Replace 'your_file.csv' with your actual CSV file path
train = np.loadtxt('train.csv', delimiter=',', skiprows=1)  # skiprows if there's a header
data_modified = np.delete(train, 0, axis=1)

In [41]:
train=pd.read_csv("train.csv")
# test=pd.read_csv('test.csv')
x=train.drop(['rainfall','id'],axis=1).copy()
y=train['rainfall']
x.head(10)

,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed
0,1,1017.4,21.2,20.6,19.9,19.4,87.0,88.0,1.1,60.0,17.2
1,2,1019.5,16.2,16.9,15.8,15.4,95.0,91.0,0.0,50.0,21.9
2,3,1024.1,19.4,16.1,14.6,9.3,75.0,47.0,8.3,70.0,18.1
3,4,1013.4,18.1,17.8,16.9,16.8,95.0,95.0,0.0,60.0,35.6
4,5,1021.8,21.3,18.4,15.2,9.6,52.0,45.0,3.6,40.0,24.8
5,6,1022.7,20.6,18.6,16.5,12.5,79.0,81.0,0.0,20.0,15.7
6,7,1022.8,19.5,18.4,15.3,11.3,56.0,46.0,7.6,20.0,28.4
7,8,1019.7,15.8,13.6,12.7,11.8,96.0,100.0,0.0,50.0,52.8
8,9,1017.4,17.6,16.5,15.6,12.5,86.0,100.0,0.0,50.0,37.5
9,10,1025.4,16.5,14.4,12.0,8.6,77.0,84.0,1.0,50.0,38.3


In [43]:

SPLIT_TIME=1752
def train_val_split(time, x,y):
    """ Splits time series into train and validations sets"""
    ttr = time[:SPLIT_TIME]
    tte = time[SPLIT_TIME:]
    xtr = x[:SPLIT_TIME]
    xte = x[SPLIT_TIME:]
    ytr = y[:SPLIT_TIME]
    yte = y[SPLIT_TIME:]

    return ttr,tte,xtr,xte,ytr,yte
ttr,tte,xtr,xte,ytr,yte = train_val_split(x['day'],x.drop(columns='day').copy(),y)

In [ ]:
# def windowed_dataset(series, window_size,batch_size,shuffle_buffer_size):
#     """Creates windowed dataset"""
#     series = tf.expand_dims(series, axis=-1)
#     dataset = tf.data.Dataset.from_tensor_slices(series)
#     dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
#     dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
#     if shuffle_buffer_size != None:
#       dataset = dataset.shuffle(shuffle_buffer_size)
#     dataset = dataset.map(lambda window: (window[:-1], window[-1]))
#     dataset = dataset.batch(batch_size).prefetch(1)
#     return dataset

# window_size = 4
# batch_size = 4
# shuffle_buffer_size = 2
# train_set = windowed_dataset(series_train, window_size, batch_size, shuffle_buffer_size)
# count=0
# for i in train_set:
#   print(i)
#   count+=1
#   if count ==5:
#     break


In [50]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
xtr = scaler.fit_transform(xtr)
# xva = scaler.transform(xva)
xte = scaler.transform(xte)

In [51]:
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, LeakyReLU, GRU
from tensorflow.keras.models import Model
import tensorflow as tf

# Model hyperparameters
input_dim =xtr.shape[1]
dropout_rate = 0.3
learning_rate = 0.001
batch_size = 32
epochs = 100
patience = 20

# Reshape input
xtr = xtr.reshape((xtr.shape[0], 1, xtr.shape[1]))
# X_val = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))
xte = xte.reshape((xte.shape[0], 1, xte.shape[1]))

# Define inputs
inputs = Input(shape=(xtr.shape[1], xtr.shape[2]))


# First LSTM layer (unidirectional)
x = LSTM(256, activation='tanh', return_sequences=True)(inputs)
x = Dropout(0.3)(x)
# Second LSTM layer
x = LSTM(128, activation='tanh', return_sequences=True)(x)
x = Dropout(0.3)(x)

# Third LSTM layer
x = LSTM(64, activation='tanh', return_sequences=True)(x)
x = Dropout(0.3)(x)

# Fourth LSTM layer (no return_sequences)
x = LSTM(32, activation='tanh')(x)

# Additional dense layer
x = Dense(16, activation='tanh')(x)

# Output layer
outputs = Dense(1, activation='sigmoid')(x)

# Build and compile model
model = Model(inputs, outputs)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])

# Summary
model.summary()

Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_9 (InputLayer)           │ (None, 1, 10)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_18 (LSTM)                       │ (None, 1, 256)              │         273,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1, 256)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_19 (LSTM)                       │ (None, 1, 128)              │         197,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 1, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_20 (LSTM)                       │ (None, 1, 64)               │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 1, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_21 (LSTM)                       │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 16)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 532,897 (2.03 MB)

 Trainable params: 532,897 (2.03 MB)

 Non-trainable params: 0 (0.00 B)

In [34]:
# Define callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=patience,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=patience//2,
    min_lr=1e-6,
    verbose=1
)

# Train model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=epochs,
    batch_size=batch_size,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

[2.0000e+00 1.0195e+03 1.6200e+01 1.6900e+01 1.5800e+01 1.5400e+01
 9.5000e+01 9.1000e+01 0.0000e+00 5.0000e+01 2.1900e+01 1.0000e+00]


In [ ]:
# Evaluate model
y_prob = model.predict(X_test).ravel()
threshold = 0.5
y_pred = (y_prob >= threshold).astype(int)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_prob)
conf_matrix = confusion_matrix(y_test, y_pred)

# Print evaluation results
print("\nModel Performance Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"AUC-ROC: {auc:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)

In [ ]:
# Preprocess test data
test_features = test_df.drop(columns=['id'], errors='ignore')  # Drop unnecessary columns
test_features = scaler.transform(test_features)
test_features = test_features.reshape((test_features.shape[0], 1, test_features.shape[1]))
test_df['rainfall'] = model.predict(test_features)
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
# Create submission file
submission = test_df[['id', 'rainfall']]  # Include 'id' and the predicted target column
submission.to_csv('submission.csv', index=False)

print("Submission file created: submission.csv")

In [36]:

example_batch = series_train.take(1)
uncompiled_model = create_uncompiled_model()
print(series_train[1])
try:
	predictions = uncompiled_model.predict(example_batch, verbose=True)
except:
	print("Your model is not compatible with the dataset you defined earlier. Check that the loss function and last layer are compatible with one another.")
else:
	print("Your current architecture is compatible with the windowed dataset! :)")
	print(f"predictions have shape: {predictions.shape}")

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_8 (Conv1D)                    │ (None, 4, 32)               │           1,280 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_16 (LSTM)                       │ (None, 4, 32)               │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_17 (LSTM)                       │ (None, 32)                  │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 17,953 (70.13 KB)

 Trainable params: 17,953 (70.13 KB)

 Non-trainable params: 0 (0.00 B)

[2.0000e+00 1.0195e+03 1.6200e+01 1.6900e+01 1.5800e+01 1.5400e+01
 9.5000e+01 9.1000e+01 0.0000e+00 5.0000e+01 2.1900e+01 1.0000e+00]
Your model is not compatible with the dataset you defined earlier. Check that the loss function and last layer are compatible with one another.
